In [ ]:
import warnings
import pandas as pd
import pm4py
import os

from datetime import datetime
from collections import defaultdict
from cfg import config as cfg
from pm4py.algo.evaluation.earth_mover_distance import algorithm as earth_mover_distance
from lib.performance_analysis import extract_ef_performance, calc_output_rate, find_diff
from lib.pre_processing import get_records
from lib.utils import print_templates, export_log_to_csv, get_event_log, convert_to_interval_tree, extract_actions_from_paths, get_test_case_directory
from lib.visualization import visualize_efg_w_freq, visualize_efg_w_duration, display_max_occurrences_in_case

warnings.filterwarnings('ignore')


In [ ]:
simulation_mode = cfg.getboolean('Mode', 'simulation')
is_preprocessing_needed = cfg.getboolean('Preprocessing', 'is_needed')

## Pre-Processing

In [ ]:
if is_preprocessing_needed:
    if simulation_mode:  
        scenario_group = "7_no_degradations"
        base_test_case_folder = "TC1_baseline_scenario"
        test_case_folder = "TC30_no_degradation_different_seed"
        home_directory = "Path/To/Test/Cases/Folder"
        log_testcases_folder = get_test_case_directory(home_directory, auto_detect=True)
    else:
        log_testcases_folder = "Path/To/primary/folder/of/the/logs"
        base_log_folder = '1_883944'
        after_change_log_folder = '2_874242'
else:
    #will be used for the results folder
    log_testcases_folder = "Path/To/primary/folder/of/the/logs"
    #should be in CSV format and include the following columns:timestamp, resource (component/module), req_id,	msg, pid (process pid if available),	function, line
    event_log_b = "Path/To/Log"
    event_log_a = "Path/To/Log"
    
   


In [ ]:
if is_preprocessing_needed:
    if simulation_mode:
        event_log_path_b = os.path.join(log_testcases_folder, scenario_group, base_test_case_folder)       
        event_log_path_a = os.path.join(log_testcases_folder, scenario_group, test_case_folder)
        filename_b = [filename for filename in os.listdir(event_log_path_b) if filename.endswith('.log')][0]
        filename_a = [filename for filename in os.listdir(event_log_path_a) if filename.endswith('.log')][0]
        raw_event_log_b = os.path.join(event_log_path_b, filename_b)
        event_log_b = os.path.join(event_log_path_b,filename_b.replace('.log', '.csv'))
        raw_event_log_a = os.path.join(event_log_path_a, filename_a)
        event_log_a = os.path.join(event_log_path_a, filename_b.replace('.log', '.csv'))
    else: 
        event_log_path_b = os.path.join(log_testcases_folder,base_log_folder)
        event_log_path_a = os.path.join(log_testcases_folder, after_change_log_folder)
        raw_event_log_b = os.path.join(event_log_path_b, 'screen-g-api.txt')
        event_log_b = os.path.join(event_log_path_b,'event_log_base.csv')
        raw_event_log_a = os.path.join(event_log_path_a, 'screen-g-api.txt')
        event_log_a = os.path.join(event_log_path_a, 'event_log_after_change.csv')
            
    # remove files if exist:
    for file in [event_log_b, event_log_a]:
        if os.path.isfile(file):
            os.remove(file)
    
    # analyze event log before commit
    with open(raw_event_log_b, "r") as fh:
        records_b, used_templates_b, changed_templates_b, templates_to_records_b = get_records(fh, [])        
        print(f"Base event log contains {len(records_b)} records, {len(used_templates_b)} recorded activities")
        
    
    with open(raw_event_log_a, "r") as fh:
        records_a, used_templates_a, changed_templates_a, templates_to_records_a = get_records(fh, used_templates_b)        
        print(f"Event log after code change contains {len(records_a)} records, {len(used_templates_a)} recorded activities")
        
    export_log_to_csv(records_b, event_log_b, True)
    export_log_to_csv(records_a, event_log_a, True)


## Detect Performance Degradations 

<span style="color:green;font-size:24px">**Extract EFRs from CSV logs**</span> 

In [ ]:
method_start_time=datetime.now()
#read CSV files
csv_df_before, event_log_before = get_event_log(event_log_b)
csv_df_after, event_log_after = get_event_log(event_log_a)

#extract EFRs
efg_b = pm4py.discover_eventually_follows_graph(event_log_before, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
efg_a = pm4py.discover_eventually_follows_graph(event_log_after, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')

In [ ]:
visualize = cfg.getboolean('Visualization', 'visualize')
if visualize:
    visualize_efg_w_freq(efg_b)
    visualize_efg_w_freq(efg_a)
    

<span style="color:green;font-size:24px">**Extract EFRs performance and variants**</span> 

In [ ]:
ef_pairs_b, variants_dfg_b = extract_ef_performance(efg_b, event_log_before)
ef_pairs_a, variants_dfg_a = extract_ef_performance(efg_a, event_log_after)

if visualize:
    visualize_efg_w_duration(ef_pairs_b)
    visualize_efg_w_duration(ef_pairs_a)

<span style="color:green;font-size:24px">**Find degradations by comparing EFRs duration**</span> 

In [ ]:
threshold = cfg.getfloat('Detect', 'threshold')
degradation_dict, improvement_dict = find_diff(ef_pairs_b, ef_pairs_a, variants_dfg_b, variants_dfg_a, threshold=threshold, print_head=5, visualize=visualize)


In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# If no degradation found, exit
if len(degradation_dict) == 0:
    folder, first_case = os.path.split(event_log_path_b)
    second_case = os.path.split(event_log_path_a)[-1]
    res_path = os.path.join(folder, "Results") 
    if not os.path.isdir(res_path):
        os.mkdir(res_path)
    with open(os.path.join(res_path, f"RES_{first_case}_{second_case}.txt"), "w") as fh:
        fh.write(f"====Classification====\n")
        fh.write('No degradations found')
    raise StopExecution

## Analyze sources for degradation

<span style="color:green;font-size:24px">**Extract activities that may affect the degradation**</span>

In [ ]:
#Extract all actions in the degraded paths
degraded_paths = dict(sorted(degradation_dict.items(), key=lambda item: item[1]["diff"], reverse=True)).keys()
export_degradations = cfg.getboolean('Export', 'degraded_paths_xes')
degradation_actions_b = extract_actions_from_paths(degraded_paths, event_log_before, event_log_path_b, export_degradations, "degraded_")
degradation_actions_a = extract_actions_from_paths(degraded_paths, event_log_after, event_log_path_a, export_degradations, "degraded_")

degradation_actions = degradation_actions_b | degradation_actions_a

In [ ]:
#Extract all actions in the improved paths
improved_paths = dict(sorted(improvement_dict.items(), key=lambda item: item[1]["diff"], reverse=True)).keys()
export_improvements = cfg.getboolean('Export', 'improved_paths_xes')
improvement_actions_b = extract_actions_from_paths(improved_paths, event_log_before, event_log_path_b, export_improvements, "improved_")
improvement_actions_a = extract_actions_from_paths(improved_paths, event_log_after, event_log_path_a, export_improvements, "improved_")

improvement_actions = improvement_actions_b | improvement_actions_a

<span style="color:green;font-size:24px">**Convert to interval tree**</span>

In [ ]:
#Extract logs into interval tree for accumulated work queues identifications 
interval_step = cfg.getint('Metric', 'period')
df_before, intervals_before = convert_to_interval_tree(event_log_before, used_templates_b, interval_step=interval_step, print_head=None)
df_after, intervals_after = convert_to_interval_tree(event_log_after, used_templates_a, interval_step=interval_step, print_head=None)
system_cases_before = len(event_log_before.groupby(['case_id']).size())
system_rate_before = system_cases_before/len(intervals_before)
print(f"system before, TP rate:{system_rate_before}, Num of cases:{system_cases_before}, intervals before:{len(intervals_before)}")
system_cases_after = len(event_log_after.groupby(['case_id']).size())
system_rate_after = system_cases_after/len(intervals_after)
print(f"system after, TP rate:{system_rate_after}, Num of cases:{system_cases_after}, intervals after:{len(intervals_after)}")
        

<span style="color:green;font-size:24px">**Calculate metrics**</span>

In [ ]:
#Padd df_before with index's that appear only in the df_after (newly introduced actions)
actions_only_in_df_after = [idx for idx in df_after.index if idx not in df_before.index]
df_before=pd.concat([df_before, pd.Series(index=actions_only_in_df_after)], axis=0).fillna(0)

#Padd df_after with index's that appear only in the df_before (removed actions)
actions_only_in_df_before = [idx for idx in df_before.index if idx not in df_after.index]
df_after=pd.concat([df_after, pd.Series(index=actions_only_in_df_before)], axis=0).fillna(0)

observed_actions = degradation_actions | improvement_actions
df_output_b = calc_output_rate(observed_actions, intervals_before, df_before, event_log_before, suffix="v1")
df_output_a = calc_output_rate(observed_actions, intervals_after, df_after, event_log_after, suffix="v2")

# concatenate mesures of the two versions 
df_output_rate = pd.concat([df_output_b, df_output_a], axis=1)



<span style="color:green;font-size:24px">**Calculate difference in metrics between software versions**</span>

In [ ]:
# Check Frequency trend
df_output_rate['Freq Diff'] = (df_output_rate['absolute freq v2'].astype(float) - df_output_rate['absolute freq v1'].astype(float))/ df_output_rate['absolute freq v1'].astype(float) 

# Check queue trend
df_output_rate['Max Queue Diff'] = (df_output_rate['max queue v2'].astype(float) - df_output_rate['max queue v1'].astype(float))/ df_output_rate['max queue v1'].astype(float)
# On the flow start activities, we cannot measure queue since there is not preceding
df_output_rate['Max Queue Diff'].fillna(0, inplace=True)

# Check TP trend
df_output_rate['TP Diff'] = (df_output_rate['TP rate v2'].astype(float) - df_output_rate['TP rate v1'].astype(float))/ df_output_rate['TP rate v1'].astype(float)
#in case that 'TP Rate v1' is 0, the above will cause na in the TP Diff
df_output_rate.loc[df_output_rate['TP Diff'].isna(), 'TP Diff'] = df_output_rate['TP rate v2']

<span style="color:green;font-size:24px">**Add max occurrences in case metric**</span>

In [ ]:
activity_counts_v1 = event_log_before.groupby(['case_id', 'msg']).size().reset_index(name='count')
msg_stats_v1 = activity_counts_v1.groupby('msg')['count'].agg(['max', 'mean', 'median'])
msg_stats_v1['mean'] = msg_stats_v1['mean'].round(2)
msg_stats_v1 = msg_stats_v1[msg_stats_v1.index.isin(observed_actions)]
msg_stats_v1['Version'] = 'v1'
df_output_rate['max occurrences in case v1'] = msg_stats_v1['max']
# fill 0 were activities belongs only to the other log 
df_output_rate['max occurrences in case v1'].fillna(0, inplace=True)

activity_counts_v2 = event_log_after.groupby(['case_id', 'msg']).size().reset_index(name='count')
msg_stats_v2 = activity_counts_v2.groupby('msg')['count'].agg(['max', 'mean', 'median'])
msg_stats_v2['mean'] = msg_stats_v2['mean'].round(2)
msg_stats_v2 = msg_stats_v2[msg_stats_v2.index.isin(observed_actions)]
msg_stats_v2['Version'] = 'v2'
df_output_rate['max occurrences in case v2'] = msg_stats_v2['max']
# fill 0 were activities belongs only to the other log
df_output_rate['max occurrences in case v2'].fillna(0, inplace=True)

df_output_rate['Max Occurrences In Case Diff'] = (df_output_rate['max occurrences in case v2'].astype(float) - df_output_rate['max occurrences in case v1'].astype(float))/df_output_rate['max occurrences in case v1'].astype(float)

if visualize:
    display_max_occurrences_in_case(msg_stats_v1, msg_stats_v2)


### Calculate Earth Movers Distance

### log before - log after

In [ ]:
emd_dict = defaultdict(list)
efr_for_analysis = defaultdict(set)

for efr in (ef_pairs_a.keys() | ef_pairs_b.keys()):
    if efr[1] in observed_actions:
        efr_for_analysis[efr[1]].add(efr)
emd_dict = defaultdict(list)

for activity, preceedings in efr_for_analysis.items():
    filtered_db_b = pm4py.filtering.filter_eventually_follows_relation(event_log_before, preceedings)
    filtered_event_log_b = pm4py.format_dataframe(filtered_db_b, case_id='case_id', activity_key='msg', timestamp_key='timestamp')
    language_b = pm4py.get_stochastic_language(filtered_event_log_b)

    filtered_db_a = pm4py.filtering.filter_eventually_follows_relation(event_log_after, preceedings)
    filtered_event_log_a = pm4py.format_dataframe(filtered_db_a, case_id='case_id', activity_key='msg', timestamp_key='timestamp')
    language_a = pm4py.get_stochastic_language(filtered_event_log_a)

    emd_log2log = earth_mover_distance.apply(language_b, language_a)
    emd_dict['StartActivities'].append(preceedings)
    emd_dict['EndActivity'].append(activity)
    emd_dict['EMD'].append(emd_log2log)

df_emd = pd.DataFrame(emd_dict)
df_emd.set_index('EndActivity', inplace=True)
df_output_rate['Earth Movers Distance'] =df_emd['EMD']
#On the start activity of the flow, we cannot measure distance since there is not preceding
df_output_rate['Earth Movers Distance'].fillna(0, inplace=True)

### Classification

In [ ]:
classifications = pd.DataFrame(False, index=df_output_rate.index, columns=["Control Flow Change", "Retry Pattern", "Over-performing", "Under-performing"]) 

#### Path changes classification

In [ ]:
classifications["Control Flow Change"] = df_output_rate['Earth Movers Distance']>=cfg.getfloat('Classification', 'emd_threshold')

#### Retry

In [ ]:
classifications["Retry Pattern"] = df_output_rate['Max Occurrences In Case Diff']>=cfg.getfloat('Classification', 'max_occur_threshold')

#### Over and Under performing

In [ ]:
classifications["Under-performing"] = ((df_output_rate['TP Diff']<=cfg.getfloat('Classification', 'under_performing_threshold') )& (df_output_rate['Max Queue Diff']>=cfg.getfloat('Classification', 'degradation_queue_threshold'))) 
classifications["Over-performing"] = ((df_output_rate['TP Diff']>=cfg.getfloat('Classification', 'over_performing_threshold') )& (df_output_rate['Max Queue Diff']<=cfg.getfloat('Classification', 'improvement_queue_threshold'))) 

In [ ]:
res_df = classifications[(
    (classifications['Control Flow Change']==True) |
    (classifications['Retry Pattern']==True) |
    (classifications['Under-performing']==True) |
    (classifications['Over-performing']==True) 
)]


#### Prioritizing source for degradation

In [ ]:
classification = defaultdict(list)
sources = []
retry_src = classifications.index[classifications['Retry Pattern'] == True].tolist()
path_change_src = classifications.index[classifications['Control Flow Change'] == True].tolist()
underperform_src = classifications.index[classifications['Under-performing'] == True].tolist()
overperform_src = classifications.index[classifications['Over-performing'] == True].tolist()
intersection = set(retry_src) & set(path_change_src)


# when new activity introduced, the max occurrences in case diff will raise
if intersection:
    for item in intersection:
        if item in actions_only_in_df_after:
            classification['Control Flow Change'].append(item)
            retry_src.remove(item) 
# Classify Retry
if retry_src:
    for item in retry_src:
        classification['Retry Pattern'].append(item)
# Classify Path Change 
elif path_change_src:
    for item in path_change_src:
        classification['Control Flow Change'].append(item)
# Classify over-performing and under-performing
else:
    if any([overperform_src, underperform_src]):
        for item in overperform_src:
            classification['Over-performing'].append(item)
        for item in underperform_src:
            classification['Under-performing'].append(item)
    else:
        classification['Results'].append("No degradation Found")


#### Write results report

In [ ]:
method_end_time=datetime.now()
folder, first_case = os.path.split(event_log_path_b)
second_case = os.path.split(event_log_path_a)[-1]
res_path = os.path.join(folder, "Results") 
if not os.path.isdir(res_path):
    os.mkdir(res_path)
    
with open(os.path.join(res_path, f"RES_{first_case}_{second_case}.txt"), "w") as fh:
    fh.write(f"====Classification====\n")
    for k,v in classification.items():
        fh.write(f"{k}:  {set(v)}\n")
    fh.write("\n\n")
    fh.write("================================\n")
    fh.write("==== Additional Information ====\n")
    fh.write("================================\n")
    print_templates(used_templates_b, prefix="V1", fh=fh)
    print_templates(used_templates_a, prefix="V2", fh=fh)
    fh.write(f"\n\n====Activities Metrics====\n")
    fh.write(df_output_rate.to_string(header=True, index=True))
    fh.write(f"\n\n====source activities====\n")
    fh.write(res_df.to_string(header=True, index=True))
    fh.write(f"\n\n====Summary====\n")
    fh.write(f"Cases before: {system_cases_before}\n")
    fh.write(f"Cases After: {system_cases_after}\n")
    fh.write(f"Intervals before:{len(intervals_before)}\n")
    fh.write(f"Intervals after:{len(intervals_after)}\n")
    fh.write(f"EF detected degradations:{len(degradation_dict)}\n")
    fh.write(f"Number of observed activities:{len(observed_actions)}\n")
    fh.write(f"Retry Patterns:{len(retry_src)}\n")
    fh.write(f"Control Flow:{len(path_change_src)}\n")
    fh.write(f"Overperforming:{len(overperform_src)}\n")
    fh.write(f"Underperforming:{len(underperform_src)}\n")
    fh.write(f"Method time:{method_end_time-method_start_time}\n")
    
    source_activities = set(retry_src) | set(path_change_src) | set(overperform_src) | set(underperform_src)
    ef_counters = defaultdict(int)
    
    for ef in degradation_dict:
        for variant in variants_dfg_a[ef]:
            ef_path= variant[3]
            for activity in ef_path:    
                if activity in source_activities:
                    ef_counters[ef] += 1
                    break    
    fh.write(f"Total degraded EFs involve source activity: {len(ef_counters)}/{len(degradation_dict)} ({len(ef_counters)/len(degradation_dict)*100:.2f}%)\n")
    
    ef_counters = defaultdict(int)
    for ef in improvement_dict:
        for variant in variants_dfg_a[ef]:
            ef_path= variant[3]
            for activity in ef_path:    
                if activity in source_activities:
                    ef_counters[ef] += 1
                    break    
    fh.write(f"Total improved EFs involve source activity: {len(ef_counters)}/{len(improvement_dict)} ({len(ef_counters)/len(improvement_dict)*100:.2f}%)\n")
    